In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt

In [ ]:
file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/All_Developer_Metrics.csv"
df = pd.read_csv(file, low_memory=False)
pd.set_option('display.max_columns', None)
#pd.set_option('display.float_format', lambda x: '%.3f' % x)
#pd.set_option('display.max_rows', None)
df.head()

In [ ]:
header = 'ALL_developer_metrics'
repos = pd.read_csv('/Users/kadir/github-data/Repos.csv',sep=',')
repo_full_names = repos[['repo_id', 'full_name']].copy()
watched_repos = pd.read_csv('/Users/kadir/github-data/Watchers-fullname.csv',sep=',')
forked_repos = pd.read_csv('/Users/kadir/github-data/Forks-fullname.csv',sep=',')
watched_repos = pd.concat([watched_repos,forked_repos]).drop_duplicates().reset_index(drop=True)
watched_repos.set_index('full_name',inplace=True)
topn= [1,3,5,10,20]
for n in topn:
    topn = n
    topn_hit = []
    names = []
    for i in range(2,46): 
        name = df.columns[i]
        x = df.loc[:,['repo_id','user_id']]
        y = df.iloc[:, [i]]
        ratings = pd.concat([x, y], axis=1, sort=False)
        ratings.rename(columns={name:'rating'}, inplace=True)  
        ratings.replace(0,np.NaN,inplace=True)
        df2 = pd.merge(ratings, repo_full_names, on='repo_id')
        repo_matrix = df2.pivot_table(index='full_name', columns='user_id', values='rating')
        repo_matrix.replace(np.nan, 0, inplace=True)
        repo_similarity_matrix = cosine_similarity(repo_matrix)
        repo_matrix2 = repo_matrix.mask(repo_matrix==0, repo_similarity_matrix@repo_matrix.values)
        accuracy = 0
        for id in repo_matrix.columns:
            x = repo_matrix[[id]][repo_matrix[[id]].values == 0].index
            recommends = repo_matrix2[[id]].loc[x].sort_values(by=[id], ascending = 0)
            topn_recom = recommends.head(topn)
            watched = watched_repos[watched_repos.user_id == id]
            # less than (n) change algorithm to calculate topn accuracy
            if len(watched)>topn:
                hit=0
                for n in topn_recom.index:
                    if n in watched.index: #full name matches
                        hit = hit +1
                    else: # only owners or repo name matches
                        wat_owners = watched.index.str.split('/').str[0].to_list()
                        wat_reponame = watched.index.str.split('/').str[1].to_list()
                        rec_owner = n.split('/')[0]
                        if rec_owner in wat_owners:
                            hit = hit + 0.5
                        if rec_owner in wat_reponame:
                            hit = hit + 0.5

                accuracy += hit * 100 / topn
            else:
                hit=0
                for n in topn_recom.index:
                    if n in watched.index: #full name matches
                        hit = hit +1
                    else: # only owners matches
                        wat_owners = watched.index.str.split('/').str[0].to_list()
                        wat_reponame = watched.index.str.split('/').str[1].to_list()
                        rec_owner = n.split('/')[0]
                        if rec_owner in wat_owners:
                            hit = hit + 0.5
                        if rec_owner in wat_reponame:
                            hit = hit + 0.5

                accuracy += hit * 100 / len(watched)

        total_accuracy = accuracy / 100
        topn_hit.append(total_accuracy)
        names.append(name)
        display(name + ':'+ str(total_accuracy))
    topn_hit = np.round(topn_hit, 2)
    hit_text = "hit_top_"+str(topn)
    results  = pd.DataFrame({hit_text:topn_hit,'metric':names})
    filename = r'/Users/kadir/Desktop/scores_'+header+'_'+hit_text+'.csv'
    export_csv = results.to_csv (filename,index=False, header=True)

In [ ]:
def highlight_max(data, color='yellow'):

    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

In [ ]:
file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/NEW_scores_ALL_developer_metrics_hit_top_1.csv"
dftop1 = pd.read_csv(file)
dftop1.sort_values(by=['hit_top_1'], ascending = 0, inplace=True)

file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/NEW_scores_ALL_developer_metrics_hit_top_3.csv"
dftop3 = pd.read_csv(file)
dftop3.sort_values(by=['hit_top_3'], ascending = 0, inplace=True)

file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/NEW_scores_ALL_developer_metrics_hit_top_5.csv"
dftop5 = pd.read_csv(file)
dftop5.sort_values(by=['hit_top_5'], ascending = 0, inplace=True)

file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/NEW_scores_ALL_developer_metrics_hit_top_10.csv"
dftop10 = pd.read_csv(file)
dftop10.sort_values(by=['hit_top_10'], ascending = 0, inplace=True)

file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/NEW_scores_ALL_developer_metrics_hit_top_20.csv"
dftop20 = pd.read_csv(file)
dftop20.sort_values(by=['hit_top_20'], ascending = 0, inplace=True)

result = pd.merge(dftop1, dftop3, how="left", on="metric")
result = pd.merge(result, dftop5, how="left", on="metric")
result = pd.merge(result, dftop10, how="left", on="metric")
result = pd.merge(result, dftop20, how="left", on="metric")

# WE ADD THIS SCORES MANUALLY.... '!!!!!!'

#add to sun's papers metric scores. (from features_of_ref10)
#new_row = {'metric':'Sun\'s metric', 
#           'hit_top_1':8.00, 'hit_top_3':6.60, 'hit_top_5':7.10, 'hit_top_10':7.35, 'hit_top_20':6.5}
#result = result.append(new_row, ignore_index=True)

#add mean of topk scores to each metric
result['mean'] = result.mean(axis=1)
result.set_index('metric', inplace=True)
result.style.apply(highlight_max)

In [ ]:
file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/NEW2_scores_ALL_developer_metrics_hit_top_1.csv"
dftop1 = pd.read_csv(file)
dftop1.sort_values(by=['hit_top_1'], ascending = 0, inplace=True)

file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/NEW2_scores_ALL_developer_metrics_hit_top_3.csv"
dftop3 = pd.read_csv(file)
dftop3.sort_values(by=['hit_top_3'], ascending = 0, inplace=True)

file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/NEW2_scores_ALL_developer_metrics_hit_top_5.csv"
dftop5 = pd.read_csv(file)
dftop5.sort_values(by=['hit_top_5'], ascending = 0, inplace=True)

file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/NEW2_scores_ALL_developer_metrics_hit_top_10.csv"
dftop10 = pd.read_csv(file)
dftop10.sort_values(by=['hit_top_10'], ascending = 0, inplace=True)

file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/NEW2_scores_ALL_developer_metrics_hit_top_20.csv"
dftop20 = pd.read_csv(file)
dftop20.sort_values(by=['hit_top_20'], ascending = 0, inplace=True)

result = pd.merge(dftop1, dftop3, how="left", on="metric")
result = pd.merge(result, dftop5, how="left", on="metric")
result = pd.merge(result, dftop10, how="left", on="metric")
result = pd.merge(result, dftop20, how="left", on="metric")

# WE ADD THIS SCORES MANUALLY.... '!!!!!!'

#add to sun's papers metric scores. (from features_of_ref10)
#new_row = {'metric':'Sun\'s metric', 
#           'hit_top_1':8.00, 'hit_top_3':6.60, 'hit_top_5':7.10, 'hit_top_10':7.35, 'hit_top_20':6.5}
#result = result.append(new_row, ignore_index=True)

#add mean of topk scores to each metric
result['mean'] = result.mean(axis=1)
result.set_index('metric', inplace=True)
result.style.apply(highlight_max)

In [ ]:
file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/comment_similarity_scores_ALL_developer_metrics_hit_top_1.csv"
dftop1 = pd.read_csv(file)
dftop1.sort_values(by=['hit_top_1'], ascending = 0, inplace=True)

file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/comment_similarity_scores_ALL_developer_metrics_hit_top_3.csv"
dftop3 = pd.read_csv(file)
dftop3.sort_values(by=['hit_top_3'], ascending = 0, inplace=True)

file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/comment_similarity_scores_ALL_developer_metrics_hit_top_5.csv"
dftop5 = pd.read_csv(file)
dftop5.sort_values(by=['hit_top_5'], ascending = 0, inplace=True)

file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/comment_similarity_scores_ALL_developer_metrics_hit_top_10.csv"
dftop10 = pd.read_csv(file)
dftop10.sort_values(by=['hit_top_10'], ascending = 0, inplace=True)

file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/comment_similarity_scores_ALL_developer_metrics_hit_top_20.csv"
dftop20 = pd.read_csv(file)
dftop20.sort_values(by=['hit_top_20'], ascending = 0, inplace=True)

result = pd.merge(dftop1, dftop3, how="left", on="metric")
result = pd.merge(result, dftop5, how="left", on="metric")
result = pd.merge(result, dftop10, how="left", on="metric")
result = pd.merge(result, dftop20, how="left", on="metric")

# WE ADD THIS SCORES MANUALLY.... '!!!!!!'

#add to sun's papers metric scores. (from features_of_ref10)
#new_row = {'metric':'Sun\'s metric', 
#           'hit_top_1':8.00, 'hit_top_3':6.60, 'hit_top_5':7.10, 'hit_top_10':7.35, 'hit_top_20':6.5}
#result = result.append(new_row, ignore_index=True)

#add mean of topk scores to each metric
result['mean'] = result.mean(axis=1)
result.set_index('metric', inplace=True)
result.style.apply(highlight_max)